# Cleaning the Philippine Standard Geographic Code Dataset

In [ ]:
import pandas as pd
import xlrd
import re

# Import the PSGC Excel file.

The Philippine Statistics Authority publishes an updated PSGC file every quarter in the form of an Excel file. The latest link is here: https://psa.gov.ph/classification/psgc/

In [ ]:
psgc_excel = pd.read_excel("data/raw/PSGC_Publication_Sept2018.xlsx",sheet_name="PSGC")
psgc_excel.to_csv('data/raw/raw-psgc.csv.gz',encoding="utf-8",compression="gzip")

In [ ]:
psgc = pd.read_csv('data/raw/raw-psgc.csv.gz',encoding="utf-8")
psgc.info()

Convert "Code" column to a string and ensure it has leading zeros and is 9-char long.

In [ ]:
psgc.loc[:,"Code"] = psgc.Code.astype(str).str.zfill(9)

Drop unused columns:

In [ ]:
psgc = psgc.loc[:,['Code','Name','Inter-Level']]

Normalize column names

In [ ]:
psgc.columns = ['code','location','interlevel']
psgc.head()

In [ ]:
psgc['interlevel'].value_counts()

In [ ]:
psgc.head()

Create a duplicate of the original PSGC dataframe

In [ ]:
og_psgc = psgc.copy()

# Helpers

We see that a lot of the locations in the PSGC have alternate names or aliases for each location contained in parentheses. Let's create a regular expression pattern that will extract these as aliases and append these as additional rows to each subset of the data.

In [ ]:
extract_in_paren = re.compile(r'\(+([^\(\)]+)\)*')
remove_in_paren = "\(.+\)"

In [ ]:
def expand_in_paren(df):
    
    '''
    Denotes original locations
    '''
    df['original'] = True
    
    '''
    Creates a copy of the rows that contain parentheses or have aliases.
    '''
    has_paren = df[df.location.str.contains("[\(\)]")]
    has_paren['original'] = False
    
    '''
    Splits locations that contain parentheses into two elements -- what's before the parentheses, and what's within them
    Each of these items is treated as a separate possible alias and appended to the original datasete
    '''
    for i in [0,1]:
        aliases = has_paren.copy()
        aliases['location'] = has_paren.location.str.replace("\)","").str.split("\(").str.get(i).str.strip()
        df = df.append(aliases,ignore_index=True)
        
    
    return df.sort_values(by=["code","original"]).reset_index(drop=True)

## Clean regions

In [ ]:
regions = psgc[psgc['interlevel'] == 'Reg'].copy()

Alternate names inside parens so we expand those out to a new column named `alias`.

In [ ]:
regions = expand_in_paren(regions)
regions

## Clean provinces

In [ ]:
provinces = psgc[psgc['interlevel'] == 'Prov'].copy()

In [ ]:
provinces.head()

Seems normal... But let's check for parens just in case:

In [ ]:
provinces[provinces['location'].str.contains('[\(\)]')]

Sneaky alternate names!

In [ ]:
provinces = expand_in_paren(provinces)
provinces

## Clean districts

In [ ]:
districts = psgc[psgc['interlevel'] == 'Dist'].copy()

In [ ]:
districts

No one writes `NTH DISTRICT (Not a Province)` in their addresses. Let's remove these instances altogether rather than extract these as aliases.

In [ ]:
districts['location'] = (districts['location']
                         .str.replace('\(Not a Province\)', '')
                         .str.strip()
                         .str.split(',',n=1)
                         .str.get(1))

In [ ]:
districts

## Clean municipalities

In [ ]:
municipalities = psgc[psgc['interlevel'] == 'Mun'].copy()

Checking for alternate names in parentheses:

In [ ]:
municipalities[municipalities['location'].str.contains('[\(\)]')]

In some cases the words "Capital" are contained in parentheses but these are not aliases. Safe to strip!

In [ ]:
municipalities['location'] = municipalities['location'].str.replace('\(Capital\)', '').str.strip()

In [ ]:
municipalities

In [ ]:
municipalities = expand_in_paren(municipalities)
municipalities.head(30)

## Clean cities

In [ ]:
cities = psgc[psgc['interlevel'] == 'City'].copy()

In [ ]:
cities.head(30)

Here we go with the `(Capital)` thing again.

In [ ]:
cities['location'] =  cities['location'].str.replace('\(Capital\)', '').str.strip()

Checking if there are still stuff with parens:

In [ ]:
cities[cities['location'].str.contains('[\(\)]')].head()

A few alterate names!

In [ ]:
cities = expand_in_paren(cities)
cities

Now what about those `CITY` pre/suffixes?

In [ ]:
cities[cities['location'].str.contains('CITY')]

Let's strip any prefixes of "CITY OF" and suffixes of "CITY."

In [ ]:
cities['location'] = (cities['location']
 .str.replace('^.*CITY OF', '') #stripping prefixes
 .str.strip()
 .str.replace('CITY$', '') #stripping suffixes
 .str.strip())

In [ ]:
cities

## Clean sub-municipalities

Manila is the only city-slash-district that has submunicipalities.

In [ ]:
sub_municipalities = psgc[psgc['interlevel'] == 'SubMun'].copy()

In [ ]:
sub_municipalities

Nothing special!

## Clean barangays

In [ ]:
barangays = psgc[psgc['interlevel'] == 'Bgy'].copy()
barangays

We see alternate names again but notice the `(Pob.)` suffixes. A quick Google search shows that it's short for `Poblacion` which is used to denote the commercial and industrial center of a city. Let's just strip these.

In [ ]:
barangays['location'] = (barangays['location']
                         .str.replace('(\(Pob\.\))', '') #totally do away with any poblacion suffixes
                         .str.strip())
barangays['location'].head(30)

How many other barangay names contain parentheses?

In [ ]:
barangays[barangays.location.str.contains(r'[\(\)]')]

While parentheses often contain aliases, sometimes, these are not aliases but the name of the municipality in which the barangay is located. For example, barangays in the municipality of Dumalneg have the `(Dumalneg)` denoted in parentheses. We'll go ahead and extract parenthetical names as aliases for now, but we'll later remove instances in which aliases are equal to the municipality name.

In [ ]:
barangays = expand_in_paren(barangays)

Let's check for more weird characters:

In [ ]:
barangays[barangays['location'].str.contains(r'[^a-zA-Z0-9\sÑñ\(\)]')]

Lets extract the strings that follow a "Brgy No. X" as aliases.

In [ ]:
pat_barangay = re.compile('(B[gr]y. No. \d+\-?\w?),? (.+)')

In [ ]:
len(barangays[barangays.location.str.contains(pat_barangay)])

In [ ]:
def expand_barangays(df):
    
    '''
    Denotes original locations
    '''
    df['original'] = True
    
    '''
    Creates a copy of the rows that contain barangay pattern
    '''
    matches_pattern = df[df.location.str.contains(pat_barangay)]
    matches_pattern['original'] = False
    
    '''
    Splits locations that into two elements -- Brgy No X and the name that comes after it
    Each of these items is treated as a separate possible alias and appended to the original datasete
    '''
    for i in [0,1]:
        aliases = matches_pattern.copy()
        aliases['location'] = matches_pattern.location.str.extract(pat_barangay)[i]#.str.get(i).str.strip()
        aliases['location'] = aliases['location'].str.strip()
        df = df.append(aliases,ignore_index=True)
        
    return df.sort_values(by=["code","original"]).reset_index(drop=True)

In [ ]:
#print len(barangays)
barangays = expand_barangays(barangays)
#print len(barangays)

Last check!

In [ ]:
barangays.info()

In [ ]:
barangays[barangays.code == "012812026"]

## ARMM: Cotabato and Isabela City

In [ ]:
armm = psgc[psgc['interlevel'].isnull()].copy()
armm

In [ ]:
armm['location'] = armm['location'].str.replace('\(Not a Province\)', '')
armm

In [ ]:
armm['location'] = (armm['location']
 .str.replace('^.*CITY OF', '')
 .str.strip()
 .str.replace('CITY$', '')
 .str.strip())
armm

In [ ]:
armm['original'] = True
armm

## All together now

In [ ]:
merged = pd.concat([
    regions,
    provinces,
    districts,
    municipalities,
    cities,
    sub_municipalities,
    barangays,
    armm
],ignore_index=True).sort_index().fillna('')

In [ ]:
merged.info()

Are counts still correct?

In [ ]:
psgc['interlevel'].value_counts()

In [ ]:
merged['interlevel'].value_counts()

In [ ]:
merged.code.nunique(), psgc.code.nunique()

## Normalize numbers:

In [ ]:
spanish = merged[merged['location'].str.contains(' (UNO|DOS|TRES|KUATRO|SINGKO)$',case=False)].copy()

In [ ]:
spanish

In [ ]:
for i, s in enumerate([
    'Uno',
    'Dos',
    'Tres',
    'Kuatro',
    'Singko',
]):
    spanish['location'] = spanish['location'].str.replace(' {}$'.format(s), ' {}'.format(i + 1))
spanish
spanish['original'] = False
spanish

In [ ]:
roman = merged[merged['location'].str.contains('\s(X{0,3})(IX|IV|V?I{0,3})$')].copy()

In [ ]:
for i, s in enumerate('I,II,III,IV,V,VI,VII,VIII,IX,X,XI,XII,XIII,XIV,XV,XVI,XVII,XVIII,XIX,XX,XXI,XXII'.split(',')):
    roman['location'] = roman['location'].str.replace(' {}$'.format(s), ' {}'.format(i + 1))
roman['original'] = False
roman

Provide alternate names for locations with President names

In [ ]:
president = merged[merged.location.str.contains('PRES\.', flags=re.IGNORECASE)].copy()
president['location'] = president['location'].str.replace('^PRES\.', 'PRESIDENT')
president['location'] = president['location'].str.replace('^Pres\.', 'President')
president['original'] = False
president

# Add alternative names to Metro Manila

In [ ]:
metro_manila = pd.DataFrame([{"code":"130000000","interlevel":"Reg","location":"Metro Manila","original":False},
              {"code":"130000000","interlevel":"Reg","location":"Metropolitan Manila","original":False}])

metro_manila

# Add Ñ -> N as an alternate name


In [ ]:
merged[merged.location.str.contains('Las Piñas',case=False)]

In [ ]:
enye = merged[merged.location.str.contains(r'[Ññ]')].copy()
enye.head()

In [ ]:
enye['location'] = (enye['location'].str.replace('Ñ', 'N')
                    .str.replace('ñ','n'))
enye.head()

# Concat the alternates to the main dataframe

In [ ]:
clean_psgc = (pd.concat([merged, spanish, roman, president], ignore_index=True)
              .sort_values('code')
              .reset_index(drop=True))

Last check for weird stuff!

In [ ]:
clean_psgc[clean_psgc['location'].str.contains('[^a-zA-Z0-9 \-.,\']')]

We can probably still split with `&` and `/` but this is good enough for now.

# Combine the cleaned up PSGC and remove the duplicates

In [ ]:
clean_psgc.drop_duplicates(subset=['code', 'location', 'interlevel'], inplace=True)
clean_psgc.reset_index(drop=True).sort_values('code', inplace=True)

Check that we have both the original name and the alternate ones

In [ ]:
clean_psgc[clean_psgc.code.str.contains('086000000')]

In [ ]:
clean_psgc[clean_psgc.code.str.contains('012801001')]

In [ ]:
clean_psgc.info()

# Cleaning out rows in which the alternate name of the barangay was just the name of its parent municipality or city

In [ ]:
clean_psgc['municipality_code'] = clean_psgc.code.str.slice(0,6)+"000"
clean_psgc['municipality'] = clean_psgc['municipality_code'].map(municipalities[municipalities.original==True].set_index('code').location)
clean_psgc.head(10)

In [ ]:
clean_psgc['drop'] = (clean_psgc.municipality == clean_psgc.location.str.upper()) & (clean_psgc.interlevel == "Bgy")

In [ ]:
barangay_and_muni_same_name = clean_psgc.groupby('code').drop.value_counts().unstack()[False][clean_psgc.groupby('code').drop.value_counts().unstack()[False].isnull()].index
clean_psgc.loc[clean_psgc.code.isin(barangay_and_muni_same_name),"drop"] = False

In [ ]:
clean_psgc = clean_psgc.loc[clean_psgc['drop'] ==False,['code','interlevel','location','original']].reset_index(drop=True)

In [ ]:
clean_psgc[clean_psgc.code == "133900000"]

# Create aliases for Legazpi and Ozamiz

In [ ]:
zplaces = clean_psgc[clean_psgc.location.str.upper().isin(["LEGAZPI","OZAMIZ"])].copy()
zplaces.loc[:,'location'] = ["LEGASPI","OZAMIS"]
zplaces

In [ ]:
clean_psgc = clean_psgc.append(zplaces,ignore_index=True)
clean_psgc

In [ ]:
clean_psgc.to_csv('data/processed/clean-psgc.csv.gz', index=False, compression='gzip')

And we're done!